# 🏥 Limpieza de Datos de Internaciones

## 🎯 Objetivo:
  Optimizar el dataset de internaciones para su exploración clínica y la construcción de dashboards, claros y útiles para stakeholders hospitalarios.

---

## Etapas de preparación del dataframe:

## 1️⃣ Observación Inicial y Unión de DataFrames:

- Se realiza una inspección preliminar de los DataFrames disponibles (`df_egreso2020`, `df_egreso2021`, `df_egreso2022`).
- Se identifican claves comunes para realizar la unión lógica entre tablas.
- Se verifica la consistencia temporal y la unicidad de las claves.

## 2️⃣ Eliminación de Columnas Innecesarias

- Se identifican columnas irrelevantes para el análisis clínico o redundantes (por ejemplo: `des_dgi_1`, `des_dge_p`).
- Se eliminan campos sin valor analítico o que no aportan a la visualización.
- Se documenta cada columna eliminada y su justificación clínica o técnica.

## 3️⃣ Tratamiento de Valores Nulos o Faltantes

- Se analiza la proporción de nulos por columna y su impacto en el análisis.
- Se documenta cada decisión de imputación o eliminación.

## 4️⃣ Transformación y Estandarización de Columnas

- Se normalizan formatos de fecha (`YYYY-MM-DD`) y se extraen componentes útiles (`mes`, `año`).
- Se estandarizan nombres de servicios clínicos, diagnósticos y categorías etarias.


---
## Conclusión:
_Esta limpieza permite construir dashboards clínicos escalables, con KPIs relevantes y visualizaciones interactivas que facilitaran la toma de decisiones operativas y estratégicas._

---

## 1.  Carga y Exploración Inicial

In [178]:
import pandas as pd
import os
import pandas as pd
import unicodedata
import numpy as np


In [179]:
# Ruta base desde el directorio del proyecto
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(BASE_DIR, "data")

def cargar_csv(nombre_archivo):
    ruta_completa = os.path.join(DATA_DIR, nombre_archivo)
    try:
        return pd.read_csv(ruta_completa)
    except FileNotFoundError:
        print(f"⚠️ Archivo no encontrado: {ruta_completa}")
        return None


df_2020 = cargar_csv("egresos2020.csv")
df_2021 = cargar_csv("egresos2021.csv")
df_2022 = cargar_csv("egresos2022.csv")

In [180]:
# Agregamos la columna año y su respectivo año en cada dataset
df_2020["año"] = 2020
df_2021["año"] = 2021
df_2022["año"] = 2022

In [181]:
# Unimos los df
df_completo = pd.concat([df_2020, df_2021, df_2022], ignore_index=True)

In [182]:
# Observamos el df inicial
df_completo.head()

,aÒo,mes,efector,idInt,idPac,sex,edad,ser,des_ser,dgi_p,des_dgi_p,dgi_1,des_dgi_1,dgi_2,des_dgi_2,dge_p,des_dge_p,dge_1,des_dge_1,dge_2,des_dge_2,tipo_egreso,sect,des_sect,estada,año,anio
0,2020.0,4,HECA,1,6545,M,21.0,35,GUARDIA EMERGENCIA - MEDICINA URGENCIA,L03,CELULITIS,NaN,NaN,NaN,NaN,G09,SECUEL.DE ENF.INFLAM.DEL SIST.NERV.CENT.,J18.9,NEUMONIA NO ESPECIFICADA,A41.9,SEPTICEMIA NO ESPECIFICADA,Defuncion,I0,AREA DE CUIDADOS INTERMEDIOS ALTOS,239,2020,NaN
1,2020.0,3,HECA,2,12186,F,57.0,35,GUARDIA EMERGENCIA - MEDICINA URGENCIA,N19,INSUFICIENCIA RENAL NO ESPECIFICADA,NaN,NaN,NaN,NaN,R99,OTRS.CAUS.MAL DEFIN.Y LAS NO ESP.DE MORT,NaN,NaN,NaN,NaN,Defuncion,I0,AREA DE CUIDADOS INTERMEDIOS ALTOS,133,2020,NaN
2,2020.0,1,HECA,3,17335,F,60.0,35,GUARDIA EMERGENCIA - MEDICINA URGENCIA,K62.5,PROCTORRAGIA,NaN,NaN,NaN,NaN,C21.8,LESION SIT.CONTG.ANO CONDC.ANAL Y RECTO,NaN,NaN,NaN,NaN,Defuncion,I0,AREA DE CUIDADOS INTERMEDIOS ALTOS,71,2020,NaN
3,2020.0,1,HECA,4,518,M,52.0,66,UROLOGIA - URINARIAS,Y84.6,CATETERIZACIÛN URINARIA,NaN,NaN,NaN,NaN,N39.0,INFECCION VIAS URINARIAS SITIO NO ESPECI,K63.2,FISTULA DEL INTESTINO,NaN,NaN,Alta medica,I0,AREA DE CUIDADOS INTERMEDIOS ALTOS,74,2020,NaN
4,2020.0,2,HECA,5,19373,M,40.0,89,UNIDAD CORONARIA,T07,TRAUMATISMOS MULTIPLES -NO ESPECIFICADOS,NaN,NaN,NaN,NaN,T07,TRAUMATISMOS MULTIPLES -NO ESPECIFICADOS,S72,FRACTURA DEL FEMUR,NaN,NaN,Alta medica,I0,AREA DE CUIDADOS INTERMEDIOS ALTOS,86,2020,NaN


## 📋 Observaciones iniciales de la estructura del dataset:


El dataset incluye registros clínicos anonimizados de pacientes internados en instituciones de salud. Cada fila representa una internación, y las columnas están organizadas por categoría temática para facilitar su análisis, limpieza y visualización posterior en Power BI.

A continuación se presenta una tabla descriptiva de las columnas contenidas en el dataset:

| Categoría                 | Columna           | Descripción                                                                 |
|--------------------------|-------------------|-----------------------------------------------------------------------------|
| 📅 Fecha de egreso        | `año`             | Año de egreso                                                               |
|                          | `mes`             | Mes de egreso                                                               |
|  Hospital               | `efector`         | Hospital de la internación                                                 |
| 🆔 Identificadores        | `idint`           | Número de identificación de la internación                                 |
|                          | `idpac`           | Número de identificación del paciente                                       |
| 🧑‍⚕️ Datos del paciente    | `sexo`            | Sexo del paciente (`Femenino`, `Masculino`)                                 |
|                          | `edad`            | Edad del paciente en años                                                  |
| 🛌 Servicio de internación| `ser`             | Código del servicio médico que realiza la atención                         |
|                          | `des_ser`         | Descripción del servicio de internación                                     |
| 🩺 Diagnóstico de ingreso | `dgi_p`           | Código CIE10 del diagnóstico principal de ingreso                           |
|                          | `des_dgi_p`       | Descripción del diagnóstico principal de ingreso                            |
|                          | `dgi_1`           | Código CIE10 del diagnóstico secundario 1 de ingreso                        |
|                          | `des_dgi_1`       | Descripción del diagnóstico secundario 1 de ingreso                         |
|                          | `dgi_2`           | Código CIE10 del diagnóstico secundario 2 de ingreso                        |
|                          | `des_dgi_2`       | Descripción del diagnóstico secundario 2 de ingreso                         |
| 🏁 Diagnóstico de egreso  | `dge_p`           | Código CIE10 del diagnóstico principal de egreso                            |
|                          | `des_dge_p`       | Descripción del diagnóstico principal de egreso                             |
|                          | `dge_1`           | Código CIE10 del diagnóstico secundario 1 de egreso                         |
|                          | `des_dge_1`       | Descripción del diagnóstico secundario 1 de egreso                          |
|                          | `dge_2`           | Código CIE10 del diagnóstico secundario 2 de egreso                         |
|                          | `des_dge_2`       | Descripción del diagnóstico secundario 2 de egreso                          |
|  Tipo de egreso         | `tipo_egreso`     | Tipo de egreso (`Alta Médica`, `Defunción`, etc.)                          |
|  Sector hospitalario    | `sect`            | Código del sector hospitalario de egreso                                   |
|                          | `des_sect`        | Descripción del sector hospitalario de egreso                              |


## 2. Limpieza de Columnas

Para facilitar el manejo y la legibilidad del DataFrame, se realiza una estandarización de los nombres de las columnas:

- Todos los nombres se convierten a minúsculas.
- Se reemplazan los espacios por guiones bajos (`_`) para facilitar el manejo, la legibilidad y se estandarizaran los nombres de las columnas.
- Se renombran algunas columnas para reflejar mejor su contenido o simplificar su uso.


In [183]:
# Visualizando todas las columnas del df 
print(df_completo.columns.tolist())

['aÒo', 'mes', 'efector', 'idInt', 'idPac', 'sex', 'edad', 'ser', 'des_ser', 'dgi_p', 'des_dgi_p', 'dgi_1', 'des_dgi_1', 'dgi_2', 'des_dgi_2', 'dge_p', 'des_dge_p', 'dge_1', 'des_dge_1', 'dge_2', 'des_dge_2', 'tipo_egreso', 'sect', 'des_sect', 'estada', 'año', 'anio']


In [184]:
# Eliminando columnas innecesarias 
df_completo.drop(columns=["sect","aÒo", "anio", 'dgi_1', 'dgi_2','dge_1', 'dge_2','dgi_p', 'des_dge_1','des_dge_2','des_dgi_2', 'des_dgi_1','dge_p','ser'], inplace=True)

In [185]:
# Reenombrando columnas
df_completo.rename(columns={"sex": "sexo_paciente", "idInt": "id_internacion", "estada": "estadia_dias", "idPac":"id_paciente","des_dgi_p":"diagnostico_ingreso","des_dge_p":"diagnostico_egreso", "des_ser":"descripcion_servicio","edad":"edad_paciente"	}, inplace=True)

In [186]:
df_completo.head()

,mes,efector,id_internacion,id_paciente,sexo_paciente,edad_paciente,descripcion_servicio,diagnostico_ingreso,diagnostico_egreso,tipo_egreso,des_sect,estadia_dias,año
0,4,HECA,1,6545,M,21.0,GUARDIA EMERGENCIA - MEDICINA URGENCIA,CELULITIS,SECUEL.DE ENF.INFLAM.DEL SIST.NERV.CENT.,Defuncion,AREA DE CUIDADOS INTERMEDIOS ALTOS,239,2020
1,3,HECA,2,12186,F,57.0,GUARDIA EMERGENCIA - MEDICINA URGENCIA,INSUFICIENCIA RENAL NO ESPECIFICADA,OTRS.CAUS.MAL DEFIN.Y LAS NO ESP.DE MORT,Defuncion,AREA DE CUIDADOS INTERMEDIOS ALTOS,133,2020
2,1,HECA,3,17335,F,60.0,GUARDIA EMERGENCIA - MEDICINA URGENCIA,PROCTORRAGIA,LESION SIT.CONTG.ANO CONDC.ANAL Y RECTO,Defuncion,AREA DE CUIDADOS INTERMEDIOS ALTOS,71,2020
3,1,HECA,4,518,M,52.0,UROLOGIA - URINARIAS,CATETERIZACIÛN URINARIA,INFECCION VIAS URINARIAS SITIO NO ESPECI,Alta medica,AREA DE CUIDADOS INTERMEDIOS ALTOS,74,2020
4,2,HECA,5,19373,M,40.0,UNIDAD CORONARIA,TRAUMATISMOS MULTIPLES -NO ESPECIFICADOS,TRAUMATISMOS MULTIPLES -NO ESPECIFICADOS,Alta medica,AREA DE CUIDADOS INTERMEDIOS ALTOS,86,2020


## 3. Manejo de valores ausentes/ Nulos 

- Identificación y gestión de los valores nulos en el dataset. La estrategia dependerá de la columna y la cantidad de nulos.


In [187]:
#3.1 Identificación de Valores Ausentes
# Contar valores nulos por columna
missing_values = df_completo.isnull().sum()
print("\nValores nulos por columna antes del tratamiento:")
print(missing_values[missing_values > 0])

# Calcular el porcentaje de valores nulos
missing_percentage = (df_completo.isnull().sum() / len(df_completo)) * 100
print("\nPorcentaje de valores nulos por columna antes del tratamiento:")
print(missing_percentage[missing_percentage > 0])

#3.2 Tratamiento de los NaNs: Eliminación de filas en valores nulos
df_completo = df_completo.dropna()
print(f'Registros restantes: {len(df_completo)}')
print(df_completo.info())


Valores nulos por columna antes del tratamiento:
edad_paciente          1110
diagnostico_ingreso    1586
diagnostico_egreso     9884
tipo_egreso              19
dtype: int64

Porcentaje de valores nulos por columna antes del tratamiento:
edad_paciente           1.187521
diagnostico_ingreso     1.696765
diagnostico_egreso     10.574290
tipo_egreso             0.020327
dtype: float64
Registros restantes: 81767
<class 'pandas.core.frame.DataFrame'>
Index: 81767 entries, 0 to 93471
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   mes                   81767 non-null  int64  
 1   efector               81767 non-null  object 
 2   id_internacion        81767 non-null  int64  
 3   id_paciente           81767 non-null  int64  
 4   sexo_paciente         81767 non-null  object 
 5   edad_paciente         81767 non-null  float64
 6   descripcion_servicio  81767 non-null  object 
 7   diagnostico_ingreso

In [188]:
#3.2 Tratamiento de los NaNs: Eliminación de filas en valores nulos
df_completo = df_completo.dropna()
print(f'Registros restantes: {len(df_completo)}')
print(df_completo.info())

Registros restantes: 81767
<class 'pandas.core.frame.DataFrame'>
Index: 81767 entries, 0 to 93471
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   mes                   81767 non-null  int64  
 1   efector               81767 non-null  object 
 2   id_internacion        81767 non-null  int64  
 3   id_paciente           81767 non-null  int64  
 4   sexo_paciente         81767 non-null  object 
 5   edad_paciente         81767 non-null  float64
 6   descripcion_servicio  81767 non-null  object 
 7   diagnostico_ingreso   81767 non-null  object 
 8   diagnostico_egreso    81767 non-null  object 
 9   tipo_egreso           81767 non-null  object 
 10  des_sect              81767 non-null  object 
 11  estadia_dias          81767 non-null  int64  
 12  año                   81767 non-null  int64  
dtypes: float64(1), int64(5), object(7)
memory usage: 8.7+ MB
None


## 4. Transformación y Estandarización de columnas

4.1  `tipo_egreso`: 


In [189]:
# Observamos los valores únicos antes de la transformación:
print("\n Valores únicos originales en 'tipo_egreso':")
print(df_completo['tipo_egreso'].unique())

# Reemplazo de valor específico por versión estandarizada
df_completo['tipo_egreso'] = df_completo['tipo_egreso'].replace('Propia voluntad', 'Retiro Voluntario')
df_completo['tipo_egreso'] = df_completo['tipo_egreso'].replace('Alta tecnica', 'Alta medica')
# Eliminando registros no pertinentes
valores_a_excluir = ['Sin orden', 'Hospital dia']
df_completo = df_completo[~df_completo['tipo_egreso'].isin(valores_a_excluir)]

#valores únicos después de la transformación
print("\n Valores únicos luego de la transformación en 'tipo_egreso':")
print(df_completo['tipo_egreso'].unique())


 Valores únicos originales en 'tipo_egreso':
['Defuncion' 'Alta medica' 'Fuga' 'Traslado' 'Propia voluntad'
 'Alta tecnica' 'Sin orden' 'Hospital dia']

 Valores únicos luego de la transformación en 'tipo_egreso':
['Defuncion' 'Alta medica' 'Fuga' 'Traslado' 'Retiro Voluntario']


4.2  `descripcion_servicio`:

In [190]:
# Valores originales  y únicos de la columna  
print(df_completo['descripcion_servicio'].unique())

['GUARDIA EMERGENCIA - MEDICINA URGENCIA' 'UROLOGIA - URINARIAS'
 'UNIDAD CORONARIA' 'NEUROCIRUGIA' 'CIRUGIA GENERAL - CLINICA QUIRURGICA'
 'TRAUMATOLOGIA TRAUMATOLO Y ORTOP- YESO' 'CARDIOLOGIA'
 'CLINICA MEDICA - MEDICINA INTERNA' 'GINECOLOGIA' 'QUEMADOS' 'NEFROLOGIA'
 'NEUROLOGIA' 'HEMATOLOGIA' 'OFTALMOLOGIA' 'CIRUGIA PLASTICA'
 'TERAPIA INTENSIVA - CUIDADOS INTENSIVOS' 'ENFERMERIA'
 'GUARDIA AMBULATORIA MOVILES' 'CIRUGIA MAXILO FACIAL' 'SALUD MENTAL'
 'OTORRINOLARINGOLOGIA O R L' 'GENETICA - ANDROLOGIA'
 'MEDICINA GENERAL CLINICA GENERAL' 'ONCOLOGIA CANCEROLOGIA'
 'CIRUGIA BUCO MAXILO FACIAL' 'GERIATRIA' 'HEMODINAMIA'
 'ADMISION DE INTERNACION' 'NEUMONOLOGIA VIAS RESPIRATORIAS'
 'GASTROENTEROLOGIA' 'ALCOHOLISMO - DROGADICCION  MIXTO'
 'TERAPIA INTENSIVA INFANTIL' 'GUARDIA PEDIATRIA' 'HEMATOLOGIA INFANTIL'
 'CIRUGIA GENERAL INFANTIL' 'CLINICA MEDICA INFANTIL-MEDICINA INTERNA'
 'INFECTOLOGIA E INFECCIOSAS INFANTIL' 'QUEMADOS INFANTIL'
 'HOSPITAL DE DIA' 'TERAPIA INTENSIVA NEONATOLOGIC

In [191]:
mapa_servicios = {
    "GUARDIA EMERGENCIA - MEDICINA URGENCIA": "Guardia",
    "GUARDIA AMBULATORIA MOVILES": "Guardia",
    "GUARDIA PEDIATRIA": "Guardia",
    "GUARDIA OBSTETRICA": "Guardia",
    "GUARDIA NEONATOLOGICA": "Guardia",
    "GUARDIA COVID19": "Guardia",
    
    "CLINICA MEDICA - MEDICINA INTERNA": "Clínica Médica",
    "CLINICA MEDICA INFANTIL-MEDICINA INTERNA": "Clínica Médica",
    "MEDICINA GENERAL CLINICA GENERAL": "Clínica Médica",

    "CIRUGIA CARDIOVASCULAR": "Cirugía",
    "CIRUGIA GENERAL - CLINICA QUIRURGICA": "Cirugía",
    "CIRUGIA GENERAL Y GINECOLOGICA": "Cirugía",
    "CIRUGIA GENERAL INFANTIL": "Cirugía",
    "CIRUGIA BUCO MAXILO FACIAL": "Cirugía",
    "CIRUGIA MAXILO FACIAL": "Cirugía",
    "CIRUGIA PLASTICA": "Cirugía",
    "NEUROCIRUGIA": "Cirugía",

    "TRAUMATOLOGIA TRAUMATOLO Y ORTOP- YESO": "Traumatología y Ortopedia",
    "TRAUMATOLOGIA INFANTIL - ORTOPEDIA Y TRA": "Traumatología y Ortopedia",

    "TERAPIA INTENSIVA - CUIDADOS INTENSIVOS": "UTI",
    "TERAPIA INTENSIVA INFANTIL": "UTI",
    "TERAPIA INTENSIVA NEONATOLOGICA": "UTI",
    "CUIDADOS INTERMEDIOS": "UTI",
    "CUIDADOS INTERMEDIOS BAJOS": "UTI",
    "UNIDAD CORONARIA": "Unidad Coronaria",

    "OBSTETRICIA": "Obstetricia",
    "GINECOLOGIA": "Ginecología",
    "TOCOGINECOLOGIA - OBSTETRICIA Y GINECOLO": "Obstetricia",
    "CARDIOLOGIA": "Cardiología",
    "NEUMONOLOGIA VIAS RESPIRATORIAS": "Neumonología",
    "QUEMADOS": "Quemados",
    "QUEMADOS INFANTIL": "Quemados",

    "SALUD MENTAL": "Salud Mental",
    "ALCOHOLISMO - DROGADICCION  MIXTO": "Salud Mental",

    "CUIDADOS ESPECIALES NEONATOLOGICOS": "Neonatología",
    "UROLOGIA - URINARIAS": "Urología",
    "OFTALMOLOGIA": "Oftalmología",
    "FISIATRIA": "Fisiatría",
    
    # Servicios que no seran categorizables → se marcan como NNan para su futura eliminación
    "OFTALMOLOGIA": np.nan,
    "PEDIATRIA GENERAL LACTANTES": np.nan,
    "INFECTOLOGIA E INFECCIOSAS INFANTIL": np.nan,
    "GERIATRIA": np.nan,
    "CENTRO INTEGRAL DE LA AUDICION": np.nan,
    "TOXICOLOGIA INFANTIL": np.nan,
    "OTORRINOLARINGOLOGIA O R L": np.nan,
    "ADMISION DE INTERNACION": np.nan,
    "ENFERMERIA": np.nan,
    "HOSPITAL DE DIA": np.nan,
    "MEDICINA LEGAL": np.nan,
    "TRABAJO SOCIAL": np.nan,
    "GASTROENTEROLOGIA": np.nan,
    "DIAGNOSTICO POR IMAGENES": np.nan,
    "ONCOLOGIA CANCEROLOGIA": np.nan,
    "HEMODINAMIA": np.nan,
    "GENETICA - ANDROLOGIA": np.nan,
    "PATOLOGIA MAMARIA": np.nan,
    "PATOLOGIA HIDROELECTROLITICA": np.nan,
    "HEMATOLOGIA": np.nan,
    "HEMATOLOGIA INFANTIL": np.nan,
    "ADMISION DE INTERNACION": np.nan,
    
}

In [192]:
df_completo['servicio_internacion'] = df_completo['descripcion_servicio'].map(mapa_servicios)

In [193]:
# Eliminamos la columna original y la  remplazamos por la nueva , con las transformaciones realizadas
df_completo = df_completo.dropna(subset=["servicio_internacion"])

In [194]:
# Observamos los valores finales de la columna
df_completo['servicio_internacion'].unique()

array(['Guardia', 'Urología', 'Unidad Coronaria', 'Cirugía',
       'Traumatología y Ortopedia', 'Cardiología', 'Clínica Médica',
       'Ginecología', 'Quemados', 'UTI', 'Salud Mental', 'Neumonología',
       'Obstetricia', 'Neonatología', 'Fisiatría'], dtype=object)

In [195]:
#Se elimina la columna original `descripcion_servicio` luego de consolidar los valores categorizados en `tipo_servicio`.
df_completo.drop(columns=['descripcion_servicio'], inplace=True)

4.3  `sexo_paciente`:

In [196]:
df_completo['sexo_paciente'].unique()
df_completo["sexo_paciente"].value_counts()

sexo_paciente
F    45013
M    34216
D       87
Name: count, dtype: int64

In [197]:
#Eliminando valores distintos a F y M.
df_completo = df_completo[df_completo['sexo_paciente'] != "D"]

4.4 `edad_paciente`:

In [198]:
# Convertir a tipo numérico (forzando errores a NaN si hay strings o símbolos)
df_completo['edad_paciente'] = pd.to_numeric(df_completo['edad_paciente'], errors='coerce').astype("Int64")

# Verificar valores nulos o extremos
print("Valores nulos:", df_completo['edad_paciente'].isna().sum())
print("Valores fuera de rango:", df_completo[(df_completo['edad_paciente'] < 0) | (df_completo['edad_paciente'] > 95)].shape[0])

Valores nulos: 0
Valores fuera de rango: 7


In [199]:
# Observando los registros fuera de rango
df_completo.loc[(df_completo['edad_paciente'] < 0) | (df_completo['edad_paciente'] > 95)].T

,5073,8911,32250,41781,75758,83717,85584
mes,8,2,5,8,11,11,4
efector,HECA,HIC,HECA,HIC,HIC,HNVV,HRSP
id_internacion,5074,907,3277,3502,5605,5644,1379
id_paciente,21697,19963,22571,23695,772,472,22809
sexo_paciente,F,F,F,F,M,M,F
edad_paciente,96,97,96,96,107,107,201
diagnostico_ingreso,TRAUMATISMO SUPEFIC.DE LA CADERA Y MUSLO,ILEO NO ESPECIFICADO,TRASTORNO COGNOSCITIVO LEVE,DISNEA,"COLICO RENAL, NO ESPECIFICADO",TRAUMATISMOS MULTIPLES -NO ESPECIFICADOS,PARTO UNICO POR CESAREA
diagnostico_egreso,"FRACTURA DEL FÈMUR, PARTE NO ESPECIFICAD",ILEO NO ESPECIFICADO,COVID-19 VIRUS NO IDENTIFICADO,INSUFICIENCIA CARDIACA NO ESPECIFICADA,"COLICO RENAL, NO ESPECIFICADO","FRACTURA COLUMNA VERTEB,NIVEL NO ESPECIF",PARTO UNICO POR CESAREA
tipo_egreso,Defuncion,Traslado,Alta medica,Traslado,Alta medica,Alta medica,Alta medica
des_sect,AREA DE CUIDADOS INTERMEDIOS BAJOS,HIC - SHOCK ROOM (CAMILLAS),AREA DE EMERGENCIA PRIMARIA,HIC - GUARDIA - (CAMAS),HIC - SHOCK ROOM (CAMILLAS),HNVV - GUARDIA,HRSP - OBSTETRICIA


In [200]:
# Eliminamos pacientes con edad mayor a 95 años
df_completo = df_completo[df_completo['edad_paciente'] <= 95]

4.5 `diagnostico_ingreso`:

In [201]:
# Ver todos los valores únicos como lista completa
diagnosticos = df_completo['diagnostico_ingreso'].unique().tolist()

In [202]:
frecuencias = df_completo["diagnostico_ingreso"].value_counts(dropna=False)

In [203]:
# Mostrar todos los resultados sin truncamiento
'''pd.set_option("display.max_rows", None)
print(frecuencias)'''

'pd.set_option("display.max_rows", None)\nprint(frecuencias)'

In [204]:
# Creamos un diccionario para renombrar los diagnosticos de ingreso y egreso asi podran ser más legibles e intepetrables para los analisis posteriores y para los stakeholders
diccionario_normalizado = {
    "PARTO UNICO ESPONT.PRESENT.CEFALIC.VERTI": "Parto único espontáneo",
    "ANOMALIA DINAMIC.DEL TRAB.PARTO NO ESPEC": "Anomalía dinámica del trabajo de parto",
    "DOLOR ABDOMINAL Y PELVICO": "Dolor abdominal y pélvico",
    "PARTO UNICO ESPONTANEO": "Parto único espontáneo",
    "COVID-19 VIRUS IDENTIFICADO": "COVID-19",
    "OTR.DOLORES ABDOMINALES Y LOS NO ESPECIF": "Otros dolores abdominales no especificados",
    "COVID-19 VIRUS NO IDENTIFICADO": "COVID-19",
    "DIARREA Y GASTROENTERITIS DE PRESUNTO OR": "Diarrea y gastroenteritis de origen infeccioso",
    "TRABAJO DE PARTO PROLONGADO NO ESPEC.": "Trabajo de parto prolongado",
    "NEUMONIA NO ESPECIFICADA": "Neumonía",
    "RUPTURA PREMATURA DE MEMBR.S/ESP": "Ruptura prematura de membranas",
    "PARTO CON CESAREA ELECTIVA": "Cesárea electiva",
    "PARTO UNICO POR CESAREA": "Parto único por cesárea",
    "NAUSEA Y VOMITO": "Náuseas y vómitos",
    "FIEBRE": "Fiebre",
    "CONVULSIONES -NO CLASIFICADAS EN OTRA P.": "Convulsiones no clasificadas",
    "TRAUMATISMO DE LA CABEZA NO ESPECIFICADO": "Traumatismo craneal no especificado",
    "DOLOR TORACICO": "Dolor torácico",
    "DISNEA": "Disnea",
    "TRAUMATISMOS MULTIPLES -NO ESPECIFICADOS": "Traumatismos múltiples",
    "OTR.ENFERM.PULM.OBSTRUCT.CRONIC.NO ESPE.": "EPOC no especificada",
    "INFECCION VIAS URINARIAS SITIO NO ESPECI": "Infección urinaria no especificada",
    "COLICO RENAL, NO ESPECIFICADO": "Cólico renal",
    "CEFALEA": "Cefalea",
    "OTROS RECIEN NACIDOS PRETERMINO": "Recién nacidos prematuros",
    "OTROS TRAUMAT.Y LOS NO ESPEC.DE LA CABEZ": "Otros traumatismos craneales",
    "ABSCESO PERIAPICAL SIN FISTULA": "Absceso dental sin fístula",
    "ESTADO ASMATICO": "Crisis asmática",
    "ABORTO MEDICO": "Aborto médico",
    "HIPERTENSION MATERNA, NO ESPECIFICADA": "Hipertensión materna",
    "ICTERICIA NEONATAL NO ESPECIFICADA": "Ictericia neonatal",
    "APENDICITIS AGUDA": "Apendicitis aguda",
    "DOLOR ABDOMINAL LOCALIZADO PARTE SUPERIO": "Dolor abdominal en epigastrio",
    "DOLOR LUMBAR": "Dolor lumbar",
    "CELULITIS": "Celulitis",
    "CELULITIS DE OTRAS PARTES DE LOS MIEMBRO": "Celulitis",
    "DOLOR PRECORDIAL": "Dolor precordial",
    "FIEBRE NO ESPECIFICADA": "Fiebre no especificada",
    "HIPERTENSION ESENCIAL (PRIMARIA)": "Hipertensión esencial",
    "APENDICITIS AGUDA NO ESPECIFICADA": "Apendicitis aguda",
    "TRABAJO DE PARTO PREMATURO SIN PARTO": "Trabajo de parto prematuro sin parto",
    "PERDIDA DE CONOCIMIENTO-SINCOPE": "Síncope",
    "ACCID.VASC.ENC.AGUDO-NO ESP.HEM.ISQUEM.": "ACV agudo no especificado",
    "INGESTA DE MEDICAMENTOS": "Ingesta de medicamentos",
    "NEFRITIS TUBULOINTERSTICIAL AGUDA": "Nefritis aguda",
    "NEFRITIS TUBULOINT.NO ESPEC.AGUDA O CRON": "Nefritis no especificada",
    "COLECISTITIS": "Colecistitis",
    "CALC.CONDUCT.BILIAR S/COLANGITIS NI COLE": "Cálculos biliares sin complicaciones",
    "OTROS TRAS.DIENTES Y ESTRUC. DE SOSTEN": "Trastornos dentales no especificados",
    "HIPERGLICEMIA NO ESPECIFICADA": "Hiperglucemia no especificada",
    "BRONQUIOLITIS AGUDA": "Bronquiolitis aguda",
    "HISTORIA PERSONAL DE LESIÛN AUTOINFLIGID": "Antecedente de lesión autoinfligida",
    "MAREOS Y VERTIGO": "Mareos y vértigo",
    "OTROS PROCEDIMIENTOS MÈDICOS": "Otros procedimientos médicos",
    "OTROS TRASTORNOS DEL SISTEMA URINARIO": "Trastornos urinarios no especificados",
    "PANCREATITIS AGUDA.": "Pancreatitis aguda",
    "FALSO TRABAJO DE PARTO S/ESPECIFICACION": "Falso trabajo de parto",
    "LUMBAGO NO ESPECIFICADO": "Lumbago no especificado",
    "TRAUMATISMO INTRACRANEAL": "Traumatismo intracraneal",
    "DIABETES MELLITUS NO INSULINODEPENDIENTE": "Diabetes tipo 2",
    "CONTAC.C/BEB,ALIM,GRAS,ACEIT P/COC.CALIE": "Contacto con alimentos calientes",
    "DOLOR EN MIEMBRO": "Dolor en extremidad",
    "BRONQUIOLITIS AGUDA, NO ESPECIFICADA": "Bronquiolitis no especificada",
    "OTROS PROCEDIMIENTOS QUIR˙RGICOS": "Otros procedimientos quirúrgicos",
    "EPISODIO DEPRESIVO NO ESPECIFICADO": "Episodio depresivo no especificado",
    "FRACTURA DEL ANTEBRAZO": "Fractura de antebrazo",
    "CONVULSIONES FEBRILES": "Convulsiones febriles",
    "ABDOMEN AGUDO": "Abdomen agudo",
    "SEGUIMIENTO POSTPARTO, DE RUTINA": "Control postparto",
    "HERIDA DE LA CABEZA": "Herida en la cabeza",
    "ENFERM.PULM.OBSTRUCTIV.CRONIC.NO ESPECIF": "EPOC no especificada",
    "INCOMPATIBILID.ABO DEL FETO Y REC.NACID.": "Incompatibilidad ABO neonatal",
    "FRACTURA A NIVEL DE LA MUÒECA Y DE MANO": "Fractura de muñeca o mano",
    "NEUMONIA BACT.NO CLASIF.EN OTRA PARTE": "Neumonía bacteriana",
    "AGRESION C/DISP.OTRS.ARMAS DE FUEGO": "Agresión con arma de fuego",
    "ANORMALIDADES DE LA RESPIRACION": "Alteraciones respiratorias",
    "OTRAS CONVULSIONES Y LAS NO CLASIFICADAS": "Convulsiones no especificadas",
    "ULCERA CRONICA DE PIEL NO CLAS.EN O.PART": "Úlcera crónica de piel",
    "GASTRITIS NO ESPECIFICADA": "Gastritis no especificada",
    "DIABETES MELLITUS INSULINODEPENDIENTE, C": "Diabetes tipo 1",
    "COLECISTITIS NO ESPECIFICADA": "Colecistitis",
    "INSUFICIENCIA CARDIACA CONGESTIVA": "Insuficiencia cardíaca congestiva",
    "DIFICULTAD RESP.DEL REC.NACID.NO ESPEC.": "Dificultad respiratoria neonatal",
    "INFECCIÛN LOCAL DE LA PIEL Y DEL TEJIDO": "Infección cutánea local",
    "MALESTAR Y FATIGA": "Malestar general y fatiga",
    "CAIDA NO ESPECIFICADA": "Caída no especificada",
    "ABORTO ESPONT·NEO, INCOMPLETO, SIN COMPL": "Aborto espontáneo incompleto",
    "DORSALGIA NO ESPECIFICADA": "Dorsalgia no especificada",
    "OBSERVACIÛN POR SOSPECHA DE OTRAS ENFERM": "Observación por sospecha de otras enfermedades",
    "HERIDA DE LA PIERNA, PARTE NO ESPECIFICA": "Herida de la pierna",           
    "ATENCIÛN Y EXAMEN INMEDIATAMENTE DESPUÈS DOLOR EN ARTICULACION ESTERILIZACIÛN": "Dolor en articulación",      
    "HERNIA INGUINAL": "Hernia Inguinal",              
    "CUERPO EXTRAÒO EN EL TUBO DIGESTIVO":"Cuerpo extraño en el tubo digestivo",
    "ENFERM.PULM.OBSTRUCT.CRONIC.C/EXACERBAC.": "EPOC con exacerbación",
    "OTROS SINTOMAS Y SIGNOS GENERALES": "Síntomas y signos generales no especificados",
    "CALC. VESIC. BILIAR SIN COLECISTITIS": "Cálculos biliares sin colecistitis",
    "HERIDA DE REGION NO ESPECIF.DEL CUERPO": "Herida",
    "PARTO PREMATURO SIN TRAB.DE PARTO ESPONT": "Parto prematuro sin trabajo de parto espontáneo",
    "DOLO EN EL PECHO NO ESPECIFICADO": "Dolor torácico no especificado",
    "ASCITIS": "Ascitis",
    "FALSO TRAB.DE PART.ANTES 37 SEMAN.COMPL.": "Falso trabajo de parto antes de las 37 semanas",
    "CELULITIS DE LA CARA": "Celulitis",
    "DIABETES MELLITUS QUE SE ORIG.EN EMBARAZ": "Diabetes gestacional",
    "INFECC.DEL RINON EN EL EMBARAZO": "Infección renal durante el embarazo",
    "FRACTURA DE MIEMBRO INFER.,NIVEL NO ESP.": "Fractura de miembro inferior no especificado",
    "FRACTURA DE LA PIERNA, INCLUSIVE TOBILLO": "Fractura de pierna o tobillo",
    "SIFILIS CONGENITA SIN OTRA ESPECIFICAC.": "Sífilis congénita no especificada",
    "HERN.INGUIN.UNILATER.NO ESP.S/OBST./GAN": "Hernia inguinal unilateral sin obstrucción ni gangrena",
    "FRACTURA DE LA PIERNA, PARTE NO ESPECIFI": "Fractura de pierna",
    "DIABETES MELLITUS, NO ESPECIFICADA, CON": "Diabetes mellitus no especificada con complicaciones",
    "INGESTA DE TOXICOS": "Intoxicación por sustancias",
    "RUPTURA PREMATURA DE LAS MEMBRANAS": "Ruptura prematura de membranas",
    "OTR.DROGAS Y SUST.BIOLOGIC.Y LAS NO ESPE": "Uso de drogas y sustancias biológicas no especificadas",
    "INFECC.NO ESPEC.DE VIAS URINAR.EN EL EMB": "Infección urinaria no especificada en el embarazo",
    "DIABETES MELLITUS NO ESPECIFICADA": "Diabetes mellitus no especificada",
    "OTROS TRAST.FUNCIONALES DEL INTESTINO": "Trastornos funcionales intestinales no especificados",
    "ANEMIA DE TIPO NO ESPECIFICADA": "Anemia no especificada",
    "ASMA": "Asma",
    "TOS":"Tos",
    "HEMATURIA":"Hematuria",                                            
    "BRONQIOLITIS AGUD.DEBID.A VIRUS SINCITAL":"Bronquiolitis",
    "ATENCIÛN Y EXAMEN INMEDIATAMENTE DESPUÈS": "Dolor en articulación",
    "FRACTURA DE OTRAS PARTES DE LA PIERNA": " Fractura de la pierna",
    "OTR.ENF.CEREBROVASCULARES ESPECIFICADAS": "Enfermedad cerebrovascular",
    "TRAUMATISMO INTRACRANEAL NO ESPECIFICADO":"Traumatismo intracraneal",
    "COLECISTITIS AGUDA":"Colecistitis",
    "ESTERILIZACIÛN_": "Esterilización",
    "ESTERILIZACIÛN": "Esterilización",
    "HERIDAS QUE AFECT.MULTIP.REGIO.DEL CUERP":"Herida",
    "OBSTRUCCION DEL CONDUCTO BILIAR":"Obstrucción del conducto biliar",
    "Fiebre no especificada":"Fiebre",
    "FRACTURA DE LA DI·FISIS DE LA TIBIA": "Fractura de la pierna",
    "PARTO CON CESAREA DE EMERGENCIA": "Parto con cesarea de emergencia",
    "Celulitis facial":"Celulitis",
    "DOLOR EN ARTICULACION" :"Dolor en articulación"        
                   
}


In [205]:
df_completo["diagnostico_ingreso"] = df_completo["diagnostico_ingreso"].replace(diccionario_normalizado)

In [206]:
print(df_completo["diagnostico_ingreso"].value_counts())

diagnostico_ingreso
Anomalía dinámica del trabajo de parto                    5414
COVID-19                                                  3985
Dolor abdominal y pélvico                                 3979
Parto único espontáneo                                    2974
Otros dolores abdominales no especificados                1760
Diarrea y gastroenteritis de origen infeccioso            1652
Trabajo de parto prolongado                               1265
Ruptura prematura de membranas                            1024
Neumonía                                                   966
Náuseas y vómitos                                          890
Cesárea electiva                                           878
Parto único por cesárea                                    868
Fiebre                                                     857
Celulitis                                                  854
EPOC no especificada                                       829
Convulsiones no clasificadas       

In [207]:
# 1. Contar frecuencia de cada diagnóstico
frecuencias = df_completo["diagnostico_ingreso"].value_counts()

# 2. Identificar los que cumplen con el umbral mínimo
diagnosticos_validos = frecuencias[frecuencias >= 98].index

# 3. Filtrar el DataFrame conservando solo esos diagnósticos
df_completo = df_completo[df_completo["diagnostico_ingreso"].isin(diagnosticos_validos)]

In [208]:
df_completo["diagnostico_ingreso"].value_counts()

diagnostico_ingreso
Anomalía dinámica del trabajo de parto                    5414
COVID-19                                                  3985
Dolor abdominal y pélvico                                 3979
Parto único espontáneo                                    2974
Otros dolores abdominales no especificados                1760
Diarrea y gastroenteritis de origen infeccioso            1652
Trabajo de parto prolongado                               1265
Ruptura prematura de membranas                            1024
Neumonía                                                   966
Náuseas y vómitos                                          890
Cesárea electiva                                           878
Parto único por cesárea                                    868
Fiebre                                                     857
Celulitis                                                  854
EPOC no especificada                                       829
Convulsiones no clasificadas       

4.6 `diagnostico_egreso`:

In [209]:
# 1. Aplicar normalización con el diccionario
df_completo["diagnostico_egreso"] = df_completo["diagnostico_egreso"].replace(diccionario_normalizado)

# 2. Eliminar espacios extra
df_completo["diagnostico_egreso"] = df_completo["diagnostico_egreso"].str.strip()

# 3. Contar frecuencia de cada diagnóstico
frecuencias = df_completo["diagnostico_egreso"].value_counts()

# 4. Identificar los que cumplen con el umbral mínimo
diagnosticos_validos = frecuencias[frecuencias >= 98].index

# 5. Filtrar el DataFrame conservando solo esos diagnósticos
df_completo = df_completo[df_completo["diagnostico_egreso"].isin(diagnosticos_validos)]

# 6. Observamos los valores final
print(df_completo["diagnostico_egreso"].value_counts(dropna=False))

diagnostico_egreso
Parto único espontáneo                                    9225
COVID-19                                                  3548
Cesárea electiva                                          2256
Otros dolores abdominales no especificados                2008
Diarrea y gastroenteritis de origen infeccioso            1755
Neumonía                                                  1264
Dolor abdominal y pélvico                                 1096
Colecistitis                                              1077
Apendicitis aguda                                          997
Parto único por cesárea                                    927
Celulitis                                                  797
EPOC no especificada                                       754
Infección urinaria no especificada                         728
Náuseas y vómitos                                          691
Cólico renal                                               609
Traumatismo craneal no especificado 

4.7 `des_sect`:

In [210]:
#Observamos los valores originarios de la descripción de los servicios
print(df_completo['des_sect'].unique())

['AREA DE CUIDADOS INTERMEDIOS ALTOS' 'AREA DE CUIDADOS CRITICOS'
 'AREA DE CUIDADOS INTERMEDIOS BAJOS' 'AREA DE EMERGENCIA PRIMARIA'
 'HIC - SALA 1' 'HIC - SALA 2' 'HIC - SHOCK ROOM (CAMILLAS)'
 'HIC - GUARDIA - (CAMAS)' 'HIC - GUARDIA AISLAMIENTO'
 'HIC - GUARDIA (CONSULTORIOS)' 'HIC - GUARDIA PASILLO' 'HIC - SALA 3'
 'HIC - TRIAGE' 'HJBA - SALA 3' 'HJBA - SALA 2: MUJERES'
 'HJBA - SALA 1: HOMBRES' 'HJBA - INTERNACION TRANSITORIA' 'HNVV - SALA 3'
 'HNVV - SALA 2' 'HNVV - SALA 1' 'HNVV - GUARDIA' 'HNVV - SALA 4'
 'HNVV - QUEMADOS' 'HNVV - UTI' 'HNVV - GUARDIA 2'
 'HRSP - CLINICA MUJERES' 'HRSP - OBSTETRICIA' 'HRSP - GUARDIA PEDIATRICA'
 'HRSP - SHOCKROOM' 'HRSP - NEONATOLOGIA' 'HRSP - GUARDIA ADULTOS'
 'HRSP - UCIM' 'HRSP - HOSPITAL DIA GINECOLOGIA'
 'HRSP - CUIDADOS MULTIPLES' 'HRSP - CLINICA HOMBRES' 'HRSP - GINECOLOGIA'
 'HRSP - PRE PARTO' 'SECTOR GENERAL' 'ILAR' 'MM - UCIN - NEO'
 'MM - INTERMEDIO - NEO' 'MM - INTERNACION CONJUNTA - NEO'
 'MM - RECUPERACION - NEO' 'MM - CUIDADOS M

In [211]:
# Diccionario: grupo → lista de palabras clave que identifican ese grupo
grupos_sector = {
    "Guardia":           ["EMERGENCIA", "GUARDIA", "TRIAGE", "SHOCK"],
    "UTI":              ["CRITICOS", "UTI", "UCIM", "CUIDADOS MULTIPLES","INTERMEDIO", "MINIMOS", "RECUPERACION", "ALTO RIESGO","RENAL"],
    "Neonatología":         ["UCIN", "NEO", "NEONATOLOGIA"],
    "Obstetricia/Ginecología": ["OBSTETRICIA", "PREPARTO", "GINECOLOGIA"],
    "Hematología-Oncología": ["HEMATO-ONCO"],
    "Quemados":             ["QUEMADOS"],
    "Sector General": ["SALA", "HOSPITAL DIA", "CRISIS SANITARIA","SECTOR GENERAL"],
   
}

In [212]:
import re

def agrupar_sector(sector):
    for grupo, keywords in grupos_sector.items():
        # unir las palabras clave en un solo patrón regex (OR)
        pattern = r"(" + "|".join(keywords) + r")"
        if re.search(pattern, sector):
            return grupo
    return "Sector General"

In [213]:
# Creamos la nueva columna
df_completo["sector_egreso"] = df_completo["des_sect"].apply(agrupar_sector)

# vemos los valores finales para la columna sector_egreso
print(df_completo["sector_egreso"].value_counts())
df_completo.drop(columns='des_sect', inplace=True)

sector_egreso
Guardia                    17407
UTI                        12063
Sector General              8762
Obstetricia/Ginecología     6696
Neonatología                1141
Quemados                     164
Name: count, dtype: int64


In [148]:
# Observamos el df final 
df_completo.head()

,mes,efector,id_internacion,id_paciente,sexo_paciente,edad_paciente,diagnostico_ingreso,diagnostico_egreso,tipo_egreso,estadia_dias,año,servicio_internacion,sector_egreso
4,2,HECA,5,19373,M,40,Traumatismos múltiples,Traumatismos múltiples,Alta medica,86,2020,Unidad Coronaria,UTI
6,1,HECA,7,19390,M,18,Traumatismo intracraneal,Traumatismo intracraneal,Defuncion,63,2020,Cirugía,UTI
10,1,HECA,11,8008,F,15,Pancreatitis aguda,Pancreatitis aguda,Alta medica,45,2020,Cirugía,UTI
18,1,HECA,19,19438,M,25,Traumatismos múltiples,Traumatismo craneal no especificado,Traslado,35,2020,Guardia,UTI
21,3,HECA,22,18981,M,76,Asma,Asma,Alta medica,97,2020,Clínica Médica,UTI


In [226]:
# Descargamos el dataset resultante luego de la limpieza.
df_completo.to_csv("df_internaciones.csv", index=False, encoding="utf-8-sig")